## About WindTrace
WindTrace is a python open-source parametric life-cycle inventory model to create tailor-made inventories of onshore wind turbines and park fleets and assess their environmental impacts. \
**IMPORTANT**: in order to run WindTrace you must have an Ecoinvent license. WindTrace is currently compatible with Ecoinvent 3.9.1, 3.10 and 3.10.1. \
The model is only compatible with the **cutoff** model. \
More information on GitHub: https://github.com/LIVENlab/WindTrace_public

Copyright (c) 2025 LIVEN lab \
Licensed under the MIT License. See LICENSE for details.

## WindTrace step-by-step

### Outlook
#### 1. Create environment and install the requirements
#### 2. Set consts.py variables
#### 3. Create/open a brightway project
#### 4. Creating life-cycle inventories
##### 4.1 Example of application
##### 4.2 Exploring the inventories
##### 4.3 Calculating LCIA scores
##### 4.4 Another example with land use focus
##### 4.5 Creating a future wind park

### 1. Create environment and install the requirements

Open the conda console and run the following line. Be sure you are in the folder where the environment.yml file is saved.
```ruby
conda env create -f environment.yml 
```
This will create a new environment called 'windtrace_env' with all the packages you need to run this Jupyter Notebook already installed. \
Make sure to activate this environment before running this Notebook.

### 2. Set consts.py variables

**IMPORTANT**: Before running anything, in the file consts.py set the following variables: PROJECT_NAME, SPOLD_FILES, and NEW_DB_NAME.
- PROJECT_NAME: name of the brightway project to be used.
- SPOLD_FILES: folder path where you have the Spoldfiles from Ecoinvent in your local disk. ONLY IF YOU DO NOT USE THE ecoinvent_interface PACKAGE
- NEW_DB_NAME: name of the databes where all the new wind turbine inventories you creeated will be stored.

### 3. Create/open a brightway project

In [5]:
import bw2io as bi
from WindTrace_onshore import *

In [6]:
bd.projects.set_current(consts.PROJECT_NAME)
bi.bw2setup()
consts.PROJECT_NAME


Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


100%|██████████| 4709/4709 [00:00<00:00, 28145.16it/s]


12:11:07+0200 [info     ] Vacuuming database            
Created database: biosphere3
Creating default LCIA methods



ValueError: Can't understand elementary flow identifier ['biosphere3', '9990b51b-7023-4700-bca0-1a32ef921f74'] in data line (['biosphere3', '9990b51b-7023-4700-bca0-1a32ef921f74'], 1.6)

In [ ]:
spold_files = consts.SPOLD_FILES
if "cutoff391" not in bd.databases:
    ei = bi.SingleOutputEcospold2Importer(spold_files, "cutoff391", use_mp=False)
    ei.apply_strategies()
    ei.write_database()
if consts.NEW_DB_NAME not in bd.databases:
    new_db = bd.Database(consts.NEW_DB_NAME)
    new_db.register()

Alternatively, you can also use ecoinvent_interface to download and install the Ecoinvent databases that you wish among versions 3.9.1, 3.10 and 3.10.1.

In [4]:
from ecoinvent_interface import Settings, permanent_setting, EcoinventRelease, ReleaseType
import os

bd.projects.set_current(consts.PROJECT_NAME)

SYSTEM_MODEL = "cutoff"
# This will install the biosphere and technosphere for Ecoinvent versions 3.9.1, 3.10 and 3.10.1. In case you 
# only want to work with one of the versions, eliminate the others from the list
EI_RELEASE_VERSIONS = ["3.10.1"]
FILE_REPOSITORY = 'Data'  # add the path where you would like to save the Ecoinvent databases

permanent_setting("username", "Your user")  # change for your Ecoinvent username
permanent_setting("password", "Your password")  #change for your Ecoinvent password
permanent_setting("output_path", FILE_REPOSITORY)


# this will download the Ecoinvent database/s for the versions you specified and write them in your brightway25 project
my_settings = Settings()
release = EcoinventRelease(my_settings)

for release_version in EI_RELEASE_VERSIONS:

    release_dir = os.path.join(FILE_REPOSITORY, f'{release_version}_ecoSpold02')
    if not os.path.exists(release_dir):
        release.get_release(release_version, SYSTEM_MODEL, ReleaseType.ecospold)
    else:
        print(f"Release already exists at {release_dir}")
    try:
        bi.import_ecoinvent_release(release_version, SYSTEM_MODEL, use_mp=False)
        print(f"Ecoinvent {release_version} ({SYSTEM_MODEL}) imported successfully!")
    except Exception as e:
        print(f"Failed to import ecoinvent release: {e}")

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds
Graph statistics for `ecoinvent-3.10.1-biosphere` importer:
4362 graph nodes:
	emission: 4000
	natural resource: 344
	inventory indicator: 13
	economic: 5
0 graph edges:
0 edges to the following databases:
0 unique unlinked edges (0 total):




100%|██████████| 4362/4362 [00:00<00:00, 30017.76it/s]


11:40:29+0200 [info     ] Vacuuming database            
Created database: ecoinvent-3.10.1-biosphere


100%|██████████| 23523/23523 [02:45<00:00, 142.49it/s]


11:43:16+0200 [info     ] Extracted 23523 datasets in 167.02 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: convert_activ

100%|██████████| 23523/23523 [00:55<00:00, 421.14it/s]


11:44:19+0200 [info     ] Vacuuming database            
Created database: ecoinvent-3.10.1-cutoff
Ecoinvent 3.10.1 (cutoff) imported successfully!


Now the database where the turbine inventories will be installed (new_db), the biosphere database (biosphere) and the ecoinvent database (ei_cutoff) must be initialised. \
**IMPORTANT**: note that the name of the database will change depending if you used the conventional bi.bw2setup() or ecoinvent_interface to install your databases.

In [2]:
new_db = bd.Database(consts.NEW_DB_NAME)
new_db.register()


biosphere = bd.Database('ecoinvent-3.10.1-biosphere')  # change for "ecoinvent-3.10-biosphere" if you are using ecoinvent 3.10 from ecoinvent_interface
ei_cutoff = bd.Database("ecoinvent-3.10.1-cutoff") # change for "ecoinvent-3.10-cutoff" if you are using ecoinvent 3.10 from ecoinvent_interface



### 4. Creating life-cycle inventories

The function allowing to create customized inventories is **lci_wind_turbine()**, which admits the following parameters:
| Parameter                     | Description                                                                  | Default                             | Type |
|------------------------------|------------------------------------------------------------------------------|-------------------------------------|------|
| Park name                    | Name of the wind park                                                        | *Required*                          | `str` |
| Park power                   | Total power of the wind park (MW)                                            | *Required*                          | `float` |
| Number of turbines           | Number of turbines in the wind park                                          | *Required*                          | `int` |
| Park location                | Country code (ISO 3166-1 alpha-2)                                            | *Required*                          | `str` |
| Park coordinates             | Latitude and longitude (WGS84)                                               | *Required*                          | `tuple` |
| Manufacturer                 | Turbine manufacturer                                                         | `LM Wind`                           | `'LM Wind'`, `'Vestas'`, `'Siemens Gamesa'`, `'Enercon'`, `'Nordex'` |
| Rotor diameter               | Rotor diameter (m)                                                           | *Required*                          | `int` |
| Turbine rated power          | Nominal power per turbine (MW)                                               | *Required*                          | `float` |
| Hub height                   | Turbine hub height (m)                                                        | *Required*                          | `float` |
| Regression adjustment        | Steel mass as function of `D2h` or `Hub height`                              | `D2h`                               | `'D2h'`, `'Hub height'` |
| Commissioning year           | Year turbine started operation                                               | *Required*                          | `int` |
| Recycled steel share         | Recycled content share of steel                                              | Eurofer data (2012–2021)            | `float` |
| Land use permanent intensity | Permanent land transformation per MW (m²/MW)                                 | `3000`                              | `float` |
| Electricity mix steel        | Electricity mix for steel production                                         | Eurofer country mix                 | `'Norway'`, `'Europe'`, `'Poland'` *(optional)* |
| Generator type               | Gearbox and generator type                                                   | `Doubly fed induction generator`    | `'dd_eesg'`, `'dd_pmsg'`, `'gb_pmsg'`, `'gb_dfig'` *(optional)* |
| Land cover type              | Land cover before installation                                               | *Required*                          | `'crops, non-irrigated'`, `'grassland'`, `'road'`, etc. |
| End-of-life scenario         | Material recycling scenario: <br> `1`: baseline<br> `2`: optimistic<br> `3`: pessimistic | `Baseline` | `str` |
| Lifetime                     | Expected lifetime of the turbine (years)                                     | `20`                                | `int` |
| Capacity factor              | Avg. delivered electricity / theoretical max                                 | `0.24`                              | `float` |
| Attrition rate               | Annual performance degradation                                               | `0.009`                             | `float` |


Parameters without a default value must be filled. Parameters with a default value are not compulsory but can be adapted to the user needs.

### 4.1. Example of application

Let's build an imaginary 100 MW wind park ('my_wind_park') of 20 turbines, 5 MW each, in a random location in Spain. \
The manufacturer of the turbine is Siemens Gamesa, with turbines of 130 m rotor diameter, 140 m hub height and a gear-boxed double-fed induction generation drivetrain. \
The park is commissioned in 2020 and is expected to last for 25 years. \
The expected initial capacity factor is 24%. \
**IMPORTANT**: make sure the name you give to the park (park_name variable) has not been used previously in your database! This means, you cannot create two wind parks with the exact same name in the same database.

In [10]:
lci_wind_turbine(park_name='my_wind_park', 
                 park_power=100.0, 
                 number_of_turbines=20, 
                 park_location='ES', 
                 park_coordinates=(41.502, -1.126), 
                 manufacturer='Siemens Gamesa', 
                 rotor_diameter=130, 
                 turbine_power=5.0, 
                 hub_height=140, 
                 commissioning_year=2020, 
                 generator_type='gb_dfig',
                 lifetime=25,  
                 cf=0.24,
                 new_db=new_db,
                 biosphere3=biosphere,
                 cutoff391=ei_cutoff
                )

Normality test for Low alloy steel (Full Range): Declined (p=0.043)
Lognormality test for Low alloy steel (Full Range): Declined (p=0.000)
Triangular test for Low alloy steel (Full Range): Declined (p=0.002)
Uniform test for Low alloy steel (Full Range): Accepted (p=0.063)
Normality test for Low alloy steel (Short Range): Accepted (p=0.089)
Normality test for Chromium steel (Full Range): Accepted (p=0.941)
Normality test for Chromium steel (Short Range): Accepted (p=0.718)
Normality test for Cast iron (Full Range): Accepted (p=0.112)
Normality test for Aluminium (Full Range): Accepted (p=0.996)
Normality test for Aluminium (Short Range): Accepted (p=0.061)
Normality test for Copper (Full Range): Declined (p=0.043)
Lognormality test for Copper (Full Range): Declined (p=0.000)
Triangular test for Copper (Full Range): Declined (p=0.000)
Uniform test for Copper (Full Range): Declined (p=0.000)
Normality test for Epoxy resin (Full Range): Accepted (p=0.109)
Normality test for Epoxy resin (S

({'Low alloy steel': 7455354.805104659,
  'Chromium steel': 1043984.2519468605,
  'Cast iron': 1681063.4157545082,
  'Aluminium': 161845.10502487788,
  'Copper': 91393.59437802702,
  'Epoxy resin': 311104.78386133275,
  'Rubber': 27173.579357149145,
  'PUR': 50570.48514631777,
  'PVC': 135072.30653721892,
  'Fiberglass': 712338.5970491616,
  'electronics': 24041.93530003225,
  'Electrics': 76255.7139360755,
  'Lubricating oil': 31331.69045155677,
  'Low alloy steel_foundations': 2052452.3349240404,
  'Chromium steel_foundations': 171367.63623532938,
  'Concrete_foundations': 15677.101183848605,
  'Praseodymium': 0.0,
  'Neodymium': 1200.0,
  'Dysprosium': 200.0,
  'Terbium': 0.0,
  'Boron': 0.0,
  'PE': 1129.9114902604463},
 50000.0,
 462500.0)

### 4.2 Exploring the inventories

The inventories are given by unit of turbine, unit of park, and kWh (for both turbine and park). In this example, the inventoris would have the following names (and codes):

- Turbine (FU=unit): my_wind_park_single_turbine
- Turbine (FU=kWh): my_wind_park_turbine_kwh
- Park (FU=unit): my_wind_park_10.0
- Park (FU=kWh): my_wind_park_kwh

You can access them as follows:

In [3]:
# park (FU=unit)
park_unit_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_100.0')
park_kwh_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_park_kwh')
turbine_unit_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_single_turbine')
turbine_kwh_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_turbine_kwh')

Altrnatively, if your interest is only the materials of one turbine, you can access them with the following code (and name):
- Turbine materials: my_wind_park_materials

In [12]:
turbine_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_materials')
for ex in turbine_act.technosphere():
    print(f'{ex.input["name"]}: {ex["amount"]} ({ex.unit})')
# Note that steel and chromium steel appear both twice. The first appearance corresponds to the turbine, and the second one to the foundations.

market for steel, low-alloyed, 2019: 372767.74025523296 (kilogram)
market for steel, chromium steel 18/8: 52199.21259734302 (kilogram)
market for cast iron: 84053.17078772541 (kilogram)
market for aluminium, wrought alloy: 7905.839208953751 (kilogram)
market for copper, cathode: 4081.5585276110005 (kilogram)
market for epoxy resin, liquid: 15555.239193066636 (kilogram)
market for synthetic rubber: 1358.6789678574573 (kilogram)
market for polyurethane, rigid foam: 2528.5242573158885 (kilogram)
market for polyvinylchloride, bulk polymerised: 6728.101196435709 (kilogram)
market for glass fibre reinforced plastic, polyamide, injection moulded: 39178.62283770389 (kilogram)
market for electronics, for control units: 1202.0967650016125 (kilogram)
cable production, unspecified: 3812.785696803775 (kilogram)
market for lubricating oil: 1566.5845225778385 (kilogram)
market for steel, low-alloyed, 2019: 102622.61674620202 (kilogram)
market for steel, chromium steel 18/8: 8568.381811766469 (kilogra

### 4.3 Calculating LCIA scores

LCA impacts of a park or an individual turbine of the park can be calculated with lca_wind_turbine(). It returns two dictionaries: the first one being results per unit and the second one being results per kWh. Default results are given per turbine. For the whole park, you must set the parameter 'turbine' to False. ReCiPe 2016 v1.03, midpoint (H) is the default LCIA method. It can be manually changed by giving the parameter 'method' another name of a method in Brightway.

Here an example of application for the whole park. The first dictionary saves scores per FU=unit, and the second per FU=1kWh

In [4]:
lcia_scores_unit, lcia_scores_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', turbine=False, new_db=new_db)
print(f'LCIA scores for (FU = one wind park unit) {lcia_scores_unit}')
print(f'LCIA scores for (FU = one kwh) {lcia_scores_kwh}')



Functional unit: 1 wind park
LCIA methods: EF v3.1
CF: 0.24. Attrition rate: 0.009
LCIA scores for (FU = one wind park unit) {'EF v3.1': 22432491.80182545, 'EF v3.1 EN15804': 22432491.80182545}
LCIA scores for (FU = one kwh) {'EF v3.1': 0.004788138341419048, 'EF v3.1 EN15804': 0.004788138341419048}
0.001055716325309905


Alternatively, you can give it a certain set of indicators of your choice as a list with the variable 'indicators'. Example:

In [ ]:
lcia_scores_unit, lcia_scores_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', 
                               indicators=[('CML v4.8 2016 no LT', 'acidification no LT', 'acidification (incl. fate, average Europe total, A&B) no LT'), 
                                           ('Cumulative Energy Demand (CED)', 'energy resources: renewable', 'energy content (HHV)'),
                                           ('Ecological Scarcity 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
                                          ], 
                               turbine=False, new_db=new_db)
print(f'LCIA scores for (FU = one wind park unit) {lcia_scores_unit}')
print(f'LCIA scores for (FU = one kwh) {lcia_scores_kwh}')

In [16]:
bd.methods

Methods dictionary with 668 objects, including:
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'ecotoxicity: freshwater', 'freshwater aquatic ecotoxicity (FAETP inf)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'ecotoxicity: marine', 'marine aquatic ecotoxicity (MAETP inf)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'ecotoxicity: terrestrial', 'terrestrial ecotoxicity (TETP inf)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)')
	('ecoinvent-3.10.1', 'CML v4.8 2016', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements

### 4.4 Another example with land use focus

Now let's build another wind park changing a bit the land use related parameters. Let's say we know the original land where the turbine will be built is a pasture. Let's say we also know that the permanent land use intensity is smaller than the default number we use. This parameter should include the squared meters per MW of land that will be occupied by permanent access roads, transformer, substation and the turbines themselves. We will use an intensity of 1000 m2/MW this time.

In [13]:
lci_wind_turbine(park_name='my_wind_park_LandUse', 
                 park_power=100.0, 
                 number_of_turbines=20, 
                 park_location='ES', 
                 park_coordinates=(41.502, -1.126), 
                 manufacturer='Siemens Gamesa', 
                 rotor_diameter=130, 
                 turbine_power=5.0, 
                 hub_height=140, 
                 commissioning_year=2020, 
                 generator_type='gb_dfig',  
                 lifetime=25,  
                 cf=0.24,
                 land_use_permanent_intensity=1000,
                 land_cover_type='pasture',
                 new_db=new_db,
                 biosphere3=biosphere,
                 cutoff391=ei_cutoff
                )

Normality test for Low alloy steel (Full Range): Declined (p=0.043)
Lognormality test for Low alloy steel (Full Range): Declined (p=0.000)
Triangular test for Low alloy steel (Full Range): Declined (p=0.002)
Uniform test for Low alloy steel (Full Range): Accepted (p=0.063)
Normality test for Low alloy steel (Short Range): Accepted (p=0.089)
Normality test for Chromium steel (Full Range): Accepted (p=0.941)
Normality test for Chromium steel (Short Range): Accepted (p=0.718)
Normality test for Cast iron (Full Range): Accepted (p=0.112)
Normality test for Aluminium (Full Range): Accepted (p=0.996)
Normality test for Aluminium (Short Range): Accepted (p=0.061)
Normality test for Copper (Full Range): Declined (p=0.043)
Lognormality test for Copper (Full Range): Declined (p=0.000)
Triangular test for Copper (Full Range): Declined (p=0.000)
Uniform test for Copper (Full Range): Declined (p=0.000)
Normality test for Epoxy resin (Full Range): Accepted (p=0.109)
Normality test for Epoxy resin (S

SystemExit: 

C:\Users\1496051\AppData\Local\miniforge3\envs\windtrace_env\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
%tb

SystemExit: 

And compare the land-use impacts with the previous wind park we created:

In [ ]:
previous_park_score_unit, previous_park_score_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', 
                               indicators=[('EF v3.1', 'land use', 'soil quality index')
                                          ], 
                               turbine=False, new_db=new_db)
new_park_score_unit, new_park_score_kwh  = lca_wind_turbine(park_name='my_wind_park_LandUse', park_power=100.0, method='EF v3.1', 
                               indicators=[('EF v3.1', 'land use', 'soil quality index')
                                          ], 
                               turbine=False, new_db=new_db)
print(f'Land Use score previous park (FU=one park): {previous_park_score_unit}; (FU=1kWh): {previous_park_score_kwh}')
print(f'Land Use score new park(FU=one park): {new_park_score_unit}; (FU=1kWh): {new_park_score_kwh}')

### 4.5 Creating a future wind park

There are some input variables that allow to model more 'futuristic' wind turbine designs. For instance, if the turbine you are designing is going to be installed 10 years from now, you may want to consider that the steel industry (with a huge impact in wind turbines) has improved its recycling shares, and is using a cleaner electricity mix. Moreover, you can adapt the end-of-life scenario to consider optimistic recycling rates. Finally, you may want to add a higher capacity factor (due to, for instance, technichal improvements). A summary of how would this be implemented:
- Steel recycling rate improvement: recycled_share_steel=0.6 (European recycled steel share in 2021 was 0.435)
- Steel electricity mix improvement: electricity_mix_steel='Norway' (only options: 'Norway', 'Europe', 'Poland')
- Optimistic recycling rates (rr): eol_scenario=2 (eol_scenario=1 means current rr, eol_scenario=2 means optimistic rr, eol_scenario=3 means pessimistic rr)
- Higher capacity factor: cf=0.4 (This is a 40%. The European mean in 2024 was 24%)

In [ ]:
lci_wind_turbine(park_name='my_wind_park_future',
                 park_power=100.0,
                 number_of_turbines=20,
                 park_location='ES',
                 park_coordinates=(41.502, -1.126),
                 manufacturer='Siemens Gamesa',
                 rotor_diameter=130,
                 turbine_power=5.0,
                 hub_height=140,
                 commissioning_year=2030,
                 generator_type='gb_dfig',
                 lifetime=25,
                 cf=0.40,
                 eol_scenario=2,
                 electricity_mix_steel='Norway',
                 recycled_share_steel=0.6,
                 new_db=new_db,
                 biosphere3=biosphere,
                 cutoff391=ei_cutoff
                )

And compare climate change impacts with the previous turbine modelled in section 4.1 in this notebook:

In [ ]:
previous_park_score_unit, previous_park_score_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1',
                               indicators=[('EF v3.1', 'climate change', 'global warming potential (GWP100)')
                                          ],
                               turbine=False, new_db=new_db)
new_park_score_unit, new_park_score_kwh  = lca_wind_turbine(park_name='my_wind_park_future', park_power=100.0, method='EF v3.1',
                               indicators=[('EF v3.1', 'climate change', 'global warming potential (GWP100)')
                                          ],
                               turbine=False, new_db=new_db)
print(f'Climate Change score previous park (FU=one park): {previous_park_score_unit}; (FU=1kWh): {previous_park_score_kwh}')
print(f'Climate Change score new park(FU=one park): {new_park_score_unit}; (FU=1kWh): {new_park_score_kwh}')